In [23]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split,cross_val_score

In [24]:
df=pd.read_csv("SalaryData_Train(1).csv")

In [25]:
df.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            30161 non-null  int64 
 1   workclass      30161 non-null  object
 2   education      30161 non-null  object
 3   educationno    30161 non-null  int64 
 4   maritalstatus  30161 non-null  object
 5   occupation     30161 non-null  object
 6   relationship   30161 non-null  object
 7   race           30161 non-null  object
 8   sex            30161 non-null  object
 9   capitalgain    30161 non-null  int64 
 10  capitalloss    30161 non-null  int64 
 11  hoursperweek   30161 non-null  int64 
 12  native         30161 non-null  object
 13  Salary         30161 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.2+ MB


In [27]:
df.drop(['workclass','race','sex','relationship','workclass','educationno'],axis=1,inplace=True)

In [28]:
list(df)

['age',
 'education',
 'maritalstatus',
 'occupation',
 'capitalgain',
 'capitalloss',
 'hoursperweek',
 'native',
 'Salary']

In [29]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
x=[
 'education',
 'maritalstatus',
 'occupation',
 'native',
 'Salary']
for i in x:
  df[i]=le.fit_transform(df[i])



In [30]:
df

,age,education,maritalstatus,occupation,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,9,4,0,2174,0,40,37,0
1,50,9,2,3,0,0,13,37,0
2,38,11,0,5,0,0,40,37,0
3,53,1,2,5,0,0,40,37,0
4,28,9,2,9,0,0,40,4,0
...,...,...,...,...,...,...,...,...,...
30156,27,7,2,12,0,0,38,37,0
30157,40,11,2,6,0,0,40,37,1
30158,58,11,6,0,0,0,40,37,0
30159,22,11,4,0,0,0,20,37,0


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   age            30161 non-null  int64
 1   education      30161 non-null  int64
 2   maritalstatus  30161 non-null  int64
 3   occupation     30161 non-null  int64
 4   capitalgain    30161 non-null  int64
 5   capitalloss    30161 non-null  int64
 6   hoursperweek   30161 non-null  int64
 7   native         30161 non-null  int64
 8   Salary         30161 non-null  int64
dtypes: int64(9)
memory usage: 2.1 MB


In [32]:
array=df.values

In [33]:
array

array([[39,  9,  4, ..., 40, 37,  0],
       [50,  9,  2, ..., 13, 37,  0],
       [38, 11,  0, ..., 40, 37,  0],
       ...,
       [58, 11,  6, ..., 40, 37,  0],
       [22, 11,  4, ..., 20, 37,  0],
       [52, 11,  2, ..., 40, 37,  1]])

In [34]:
X=array[:,:-1]
Y=array[:,-1]


In [35]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.5)

In [36]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((15080, 8), (15081, 8), (15080,), (15081,))

In [37]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")

    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [38]:
rf_clf = SVC()
rf_clf.fit(X_train, Y_train)

print_score(rf_clf, X_train, Y_train, X_test, Y_test, train=True)
print_score(rf_clf, X_train, Y_train, X_test, Y_test, train=False)

Train Result:
Accuracy Score: 79.18%
_______________________________________________
CLASSIFICATION REPORT:
                      0            1  accuracy     macro avg  weighted avg
precision      0.796574     0.744118  0.791844      0.770346      0.783345
recall         0.969141     0.266106  0.791844      0.617623      0.791844
f1-score       0.874425     0.392020  0.791844      0.633223      0.752768
support    11277.000000  3803.000000  0.791844  15080.000000  15080.000000
_______________________________________________
Confusion Matrix: 
 [[10929   348]
 [ 2791  1012]]

Test Result:
Accuracy Score: 80.02%
_______________________________________________
CLASSIFICATION REPORT:
                      0            1  accuracy     macro avg  weighted avg
precision      0.805688     0.746790  0.800212      0.776239      0.791218
recall         0.968794     0.282591  0.800212      0.625693      0.800212
f1-score       0.879745     0.410025  0.800212      0.644885      0.764347
support   